# faiss安装
## 使用Anaconda安装
使用Anaconda安装使用faiss是最方便快速的方式，facebook会及时推出faiss的新版本conda安装包，在conda安装时会自行安装所需的libgcc, mkl, numpy模块。  
faiss的cpu版本目前仅支持Linux和MacOS操作系统，gpu版本提供可在Linux操作系统下用CUDA8.0/CUDA9.0/CUDA9.1编译的版本。  
注意，上面语句中的cuda90并不会执行安装CUDA的操作，需要提前自行安装。

In [ ]:
#安装cpu版本
#更新conda
conda update conda
#先安装mkl
conda install mkl
#安装faiss-cpu
conda install faiss-cpu -c pytorch
#测试安装是否成功
python -c "import faiss"

In [ ]:
#安装gpu版本
#确保已经安装了CUDA，否则会自动安装cup版本。
conda install faiss-gpu -c pytorch # 默认 For CUDA8.0
conda install faiss-gpu cuda90 -c pytorch # For CUDA9.0
conda install faiss-gpu cuda91 -c pytorch # For CUDA9.1

注意，上面语句中的cuda90/91并不会执行安装CUDA的操作，需要提前自行安装。

## 编译安装
编译安装需要3个步骤：  
1.编译C++文件；  
2.编译Python界面；  
3.编译GPU部分。  
步骤1是必要的，步骤2和3相互独立，不存在相互依赖。  
faiss通过一个Makefile文件编译，系统相关的配置文件包含在makefile.inc这个文件中，在下载的faiss安装文件中的example_makefiles文件夹下有实例文件，选择适合的操作系统，将其复制到faiss根目录下，并重命名为makefile.inc。文件中的部分参数需要手动修改。  
### 第一步：编译C++ faiss
#### BLAS/Lapack

In [ ]:
# ubuntu 16
sudo apt-get install libopenblas-dev python-numpy python-dev
# ubuntu 14
sudo apt-get install libopenblas-dev liblapack3 python-numpy python-dev

# 执行配置文件
./configure
#检查BLAS
make misc/test_blas
./misc/test_blas

#### 建造faiss

In [ ]:
make
make install

### 第二步 编译Python界面

In [ ]:
#安装swig
sudo apt-get install swig  # ubuntu 16
make py
# 默认使用系统python2.7，如果想使用其他版本的python,需要在makefile.inc文件中修改PYTHONCFLAGS值为python安装路径。

#### 测试faiss的python使用

In [1]:
#编译只有在faiss/python文件夹下应该包含如下三个文件
#faiss.py
#swigfaiss.py  / swigfaiss_gpu.py
#_swigfaiss.so / _swigfaiss_gpu.so

#编译不报错并不意味着faiss可以正常使用，
python -c "import faiss"
#如果不能import，可能是faiss的路径没配置正确，可以在使用时添加代码
import sys
sys.path.append(faiss_path) # 如 ‘/home/faiss/python/’
#可能是有一些symbols丢失了，使用下面的命令查看
cd python
ldd -r _swigfaiss.so
# 有时候只有在使用BLAS库的时候报错，使用下面的命令测试
python -c "import faiss, numpy"
faiss.Kmeans(10, 20).train(numpy.random.rand(1000, 10).astype('float32'))

SyntaxError: invalid syntax (<ipython-input-1-e26e3db123fe>, line 7)

### 第三步 编译GPU部分  
GPU 开发环境是 GPU-FAISS 开发的基础，主要分为安装CUDA并导入CuDNN关联、配置环境变量三部分。
参考：https://www.cnblogs.com/zealousness/p/8757290.html

#### 安装CUDA并导入CuDNN关联
CUDA， Compute Unified Device Architecture， 是 NVIDIA 推出的通用并行计算框架。CuDNN 是 NVIDIA 提供的 GPU 计算加速方案。 

#### 1.2、配置环境变量
为开发用户定制环境变量。

``` 
vim $HOME/.bash_profile

export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/usr/local/cuda/lib64"
export CUDA_HOME=/usr/local/cuda export PATH="$PATH:/usr/local/cuda/bin"


source $HOME/.bash_profile
```

#### 安装
#进入gpu目录 &编译

``` shell
cd gpu
make py
```

测试：在faiss文件夹下，进入python2命令行，执行
```python
import faiss
import _swigfaiss_gpu
```
执行 成功则安装成功。